In [ ]:
using Plots, LaTeXStrings#, DifferentialEquations, Sundials

\begin{equation}
\begin{split}
\Omega_m  &=  0.315 \pm 0.007 \\
\Omega_\Lambda  &= 0.6847 \pm 0.0073 \\
\Omega_r &= 9.4 \cdot 10^{-5} \\
\mid \Omega_\kappa \mid  &\leq 0.01 \\
H_0 [\mathrm{km}\text{ }\mathrm{s}^{-1} \mathrm{Mpc}^{-1}] &=  67.4 \pm 0.5  .
\end{split}
\end{equation}

In [ ]:
H0 = 70 # Hubble constant H = 70 km/s/Mpc
Mpc = 3.085677581e19 # km
Gyr = 3.1536e16 # second
H₀ = H0*Gyr/Mpc # in the units of 1/Gyr

$$
\Big(\frac{\dot{a}}{a}\Big)^2 = H^2 = H_0^2 \Big{[} \Omega_{r, 0}\Big(\frac{a_0}{a}\Big)^{4} + \Omega_{m, 0} \Big(\frac{a_0}{a}\Big)^{3} + \Omega_{\kappa, 0} \Big(\frac{a_0}{a}\Big)^{2} + \Omega_{\Lambda, 0}\Big{]} ,
$$

\begin{equation}
H_0dt = \int_0^{a_0} \frac{da}{\Big{[} \Omega_{r, 0}a^{-2} + \Omega_{m, 0}a^{-1} + \Omega_{\kappa, 0} + \Omega_{\Lambda, 0}a^2\Big{]}^{1/2}},
\end{equation}

In [ ]:
function Friedmann(a,m,r,l)
    k = 1.0 - (m + r + l)
    Omegas = r/(a*a) + m/a + l*a*a + k
    return 1.0 / ( H₀*sqrt(Omegas) ) 
end

function EmptyUniverse(t)
    a_e = H₀*t
    age_e = 1/H₀
    return a_e, age_e
end

https://ru.wikipedia.org/wiki/Формула_Кардано

In [ ]:
function real_cardano(p,q)
    Q = complex( (p/3)^3 + (0.5q)^2 )
    #println(Q)
    α = ( -0.5q + sqrt(Q) )^(1/3)
    β = ( -0.5q - sqrt(Q) )^(1/3)
    y1 = α + β
    y2 = -0.5(α+β) + 0.5*im*(α-β)*sqrt(3)
    y3 = -0.5(α+β) - 0.5*im*(α-β)*sqrt(3)
    ret = Inf
    Y = [y1, y2, y3]
    #println(Y)
    Y[findall(isreal, Y)] |> real |> maximum
end

$$
\Omega_{m, 0}a^{-1} + \Omega_{\kappa, 0} + \Omega_{\Lambda, 0}a^2 = 0
$$

$\Omega_0 = \sum\Omega_i$

$$
a^3 + \frac{\Omega_{\kappa, 0}}{\Omega_{\Lambda, 0}}a + \frac{\Omega_{m, 0}}{\Omega_{\Lambda, 0}} = 0
$$

In [ ]:
function big_bounce(a, m, r, l)
    
    n = 5000 # Step number
    k = 1.0 - (m + l)
    b = real_cardano(k/l, m/l)
    dx = (b-a)/n # Step size
    
    x = zeros(n)
    y = zeros(n)
    z = zeros(n)
    for i in 1:n
        x[i] = a + (i-1)*dx
        y[i] = Friedmann(x[i],m,r,l)
        z[i] = 0.5*dx * (2*sum(y) - y[1] - y[n])
    end
    return [x; reverse(x)], [z.-z[end];-reverse(z).+z[end]]
end

In [ ]:
function Trapezoidal(a,b,m,r,l)
    
    n = 20000 # Step number
    dx = (b-a)/n # Step size
    AGE = 0
    
    x = zeros(n)
    y = zeros(n)
    z = zeros(n)
    
    for i in 1:n
        x[i] = a + (i-1)*dx
        y[i] = Friedmann(x[i],m,r,l)
        z[i] = 0.5*dx * (2*sum(y) - y[1] - y[n])

        if 1-dx <= x[i] <= 1+dx
            AGE = z[i]
        end
    end
    println("Age of the universe with m = $m, r = $r, lambda = $l is $(r2(AGE)) Gyr")

    return x[1:100:end], z[1:100:end], AGE
end

r2(x) = round(x, digits = 2);

$
\Lambda < 0
$

$$
t_{crunch} = \frac{2\pi}{3H_0}\frac{1}{\sqrt{\Omega_{m,0}-1}}
$$

$$
a_{max} = \left( \frac{\Omega_{m,0}}{\Omega_{m,0}-1} \right)^{1/3}
$$
$$
H_0t = \frac{2}{3\sqrt{\Omega_{m,0}-1}}\arcsin\left[ \left(\frac{a}{a_{max}}\right)^{3/2} \right]
$$
$$
a(t) = \left[ a_{max}^{3/2}\sin\left( \frac 32 H_0 t\sqrt{\Omega_{m,0}-1} \right) \right]^{2/3}
$$
$$
t_0 = \frac{2}{3H_0\sqrt{\Omega_{m,0}-1}}\arcsin\left( \sqrt{\frac{\Omega_{m,0}-1}{\Omega_{m,0}}} \right)
$$

In [ ]:
function lambda_negative(m)
    t_cruch = 2π / (3H₀*sqrt(m-1))
    T = 0:1e-2:t_cruch
    amax32 = sqrt( m/(m-1.0) )
    a(t) = cbrt( amax32*sin( 1.5*H₀*t*sqrt(m-1) ) )^2
    t_0 = 2.0/(3.0*H₀*sqrt(m-1))*asin( sqrt(1-1/m) )
    a.(T), T, t_0
end

In [ ]:
a,t,age = Trapezoidal(1e-10,10,0.315,9.4e-5,0.6847) # Λ
a1,t1,age1 = Trapezoidal(1e-10,10,0.5,0.5,0) # M + R
a2,t2,age2 = Trapezoidal(1e-10,10,0.7,0.2,0.1) 
a3,t3,age3 = Trapezoidal(1e-10,10,0.3,0.0,1.71) # Loitering
a4,t4,age4 = lambda_negative(1.3) # Λ Collapse
a5,t5 = big_bounce(10,0.3,0.0,1.8)


t_i = 0
t_f = 90
steps = 1000
t_e = range(t_i, stop = t_f, length = steps)
a_e,age_e = EmptyUniverse(t_e);

In [ ]:
plot(t.-age, a, line = 3, color=:blue, label = L"\Lambda-CDM")
plot!(t1.-age1, a1, line = 3, color=:crimson, label = L"\Omega_{m,0} = 0.5, \Omega_{r,0} = 0.5")
plot!(t2.-age2, a2, line = 3, color=:forestgreen, 
    label = L"\Omega_{m,0} = 0.7, \Omega_{r,0} = 0.2, \Omega_{\Lambda,0} = 0.1")
plot!(t_e.-age_e,a_e, line = (2, :dash), color = :black, label = "Empty Universe")
plot!(t3.-age3, a3, line = 3, color=:orange, label = "Loitering")
plot!(t4.-age4, a4, line = 3, color=:magenta, label = L"\Lambda Collapse")
plot!(t5.-age, a5, line = 3, color=:brown, label = "Big Bounce")

xaxis!("t, Gyr", (-50,60), legend = :topleft )
yaxis!("a(t)", (0,5))

scatter!([0],[1], m = (:black, 10, 5), lab = "") # ,zorder = 10,s = 13
plot!([-60, 0, 0], [1, 1, -1], line = (2, "black", :dot ), lab = "")

In [ ]:
savefig("fateou.png")